In [9]:
from requests import get
import pandas as pd

In [ ]:
# første sted...TV2!

In [25]:
tv2_rå_svar = get("https://election-api.services.tv2.dk/ep/ep24/candidatetest/answers/999").json()

In [26]:
tv2_spg = [ '1a', '1b', '2a', '2b', '3a', '3b', '4a', '4b', '5a', '5b', '6a', '6b', '7a', '7b', '8a', '8b', '9a', '9b', '10a', '10b', '11a', '11b']

In [27]:
# smæk det hele pænt sammen

tv2_svar = {}
tv2_kandidater = {}
for kandidat in tv2_rå_svar:
    tv2_kandidater[kandidat['id']] = {'navn': kandidat['name'], 'parti': kandidat['partyName'], 'job': kandidat['occupation'], 'alder': kandidat['age']}
    tv2_svar[kandidat['id']] = {x: kandidat['answers'][x]['answer'] for x in tv2_spg if x in kandidat['answers']}
tv2_kandidater = pd.DataFrame(tv2_kandidater).T
tv2_svar = pd.DataFrame(tv2_svar).T

In [28]:
# norm mellem 0 og 1
ma = tv2_svar.max()
mi = tv2_svar.min()
tv2_svar = (tv2_svar-mi)/(ma-mi)

In [30]:
q = tv2_svar.copy()
q['parti'] = [tv2_kandidater.loc[x, 'parti'] for x in tv2_svar.index]
q['navn'] = [tv2_kandidater.loc[x, 'navn'] for x in tv2_svar.index]
q['job'] = [tv2_kandidater.loc[x, 'job'] for x in tv2_svar.index]
q['alder'] = [tv2_kandidater.loc[x, 'alder'] for x in tv2_svar.index]
#q.parti.replace("Konservative", "Det Konservative Folkeparti", inplace=True)

In [32]:
q.to_feather("2024_EU_Lasse_data.feather")

In [ ]:
https://web.tv2a.dk/bundles/3452.4e7453363cc73ec7.js